In [17]:
##################################################################################################
# fiveguys_2a.ipynb
# Team C: Five Guys: Burgers and Fries
# Aditya Jain, Alex Sandoval, Darrell Harvey, and Fasih Ahsan
# COM SCI X 450.1 Section 362062
# UCLA Extension, Spring 2018
# IPython Notebook for Class Project 2b Submission
# Script creates an EmotiFind function predictEmotion()
# Function input is a UTF-8 .txt file for any article, tweet, etc. Must be in the same folder as this .ipynb. 
# Function output is standard stream print of predicted emotion values for 8 emotions
# Sadness, Anger, Joy, Trust, Fear, Surprise, Disgust, Anticipation
# Must also keep in same folder as .ipynb: EmoLex, TrainingSetEmotions, TrainingSetArticles .txt files.
# Unzip these files from the submission. 
# Requires installation of nltk corpus, csv, NumPy, Matplotlib, SciKit-Learn
# Version 0.0.1  - 05/13/2018
####################################################################################################


# Import required libraries and packages. 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import string
from collections import defaultdict
import csv
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt

# Import default dictionary for dictionary lists, initialize list and dicts. 
#from collections import defaultdict
emotion_dict = defaultdict(list)
emolexdatalist= []
train_sadness = []
train_anger = []
train_joy = []
train_trust = []
train_fear = []
train_surprise = []
train_disgust = []
train_anticipation = []

# Open and read the EmoLex into a list of lists. 
with open("NRC-Emotion-Lexicon-Wordlevel-v0.92.txt", "r") as emolexdata:
    next(emolexdata)
    for row in emolexdata:
        row = row.strip().split("\t")
        emolexdatalist.append(row)
    emolexdata.close()

# For each list in the list of lists 
for line in emolexdatalist:
    # If the word is included in the emotion
    if line[2] == "1":
        # And if the emotion is not a sentiment
        if (line[1] != 'negative') and (line[1] != 'positive'):
            # Add the word to the list of lists
            emotion_dict[line[1]].append(line[0])
        else:
            # Otherwise if it is 0, and if is a sentiment, skip. 
            continue
            
#Open and import the training set emotions.txt
with open("TrainingSetEmotions.txt", "r") as trainingsetemotions:
    next(trainingsetemotions)
    reader = csv.reader(trainingsetemotions,delimiter='\t')
    for sadness,anger,joy,trust,fear,surprise,disgust,anticipation in reader:
        train_sadness.append(sadness)
        train_anger.append(anger)
        train_joy.append(joy)
        train_trust.append(trust)
        train_fear.append(fear)
        train_surprise.append(surprise)
        train_disgust.append(disgust)
        train_anticipation.append(anticipation)

# @function clean_article()
# @input articleName is the .txt file of the article to clean. 
def clean_article(articleName):
    token_frequency_dic = {}
    with open(articleName,'r') as f:
        text = f.read()

        # split into words
        tokens = word_tokenize(text)

        # convert to lower case
        tokens = [w.lower() for w in tokens]

        # remove punctuation from each word
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]

        # remove remaining tokens that are not alphabetic
        words = [word for word in stripped if word.isalpha()]

        # filter out stop words and sort
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        words.sort()
        req = nltk.FreqDist(words)
        for k,v in req.items():
            token_frequency_dic[str(k)] = v
               
        return token_frequency_dic

    f.close()

# @function
    #part 1 of checking the words in emption_dict
def token_checker_1(emotion_dict, token_frequency):
    token_frequency_present = {}
    token_frequency_absent = {}
    #creating a blank dictionary for each emotion based on emotion_dict
    for emotion in emotion_dict:
        token_frequency_present[emotion] = {}
        
    for word in token_frequency: #taking the word 
        present = False  #internal check to see if the word is present in any of the emotion dicts
        for emotion in emotion_dict: # going into sad in the emotion dict
            if word in emotion_dict[emotion]: # going into the list of words in sad words
                token_frequency_present[emotion][word]= token_frequency[word]
                present = True #will change if 
        if present == False:
            token_frequency_absent[word]= token_frequency[word]
    return [token_frequency_present,token_frequency_absent]



#part 2 of checking the words in the emption_dict
def token_checker_2(token_frequency_present,token_frequency_absent,emotion_dict):
    lemmatizer = WordNetLemmatizer()
    token_frequency_absent_l = {}
    for word in token_frequency_absent: #taking the word sorrow
        word_lemma = lemmatizer.lemmatize(word)
        for word_p in token_frequency_present:
            if word_lemma is word_p:
                token_frequency_present[word_p] = token_frequency_present[word_p] + token_frequency_absent[word]
            else:
                token_frequency_absent_l[word_lemma] = token_frequency_absent[word]

    [token_frequency_present_2,token_frequency_absent_2] = token_checker_1(emotion_dict,token_frequency_absent_l)

    for emotion in token_frequency_present_2:
        if emotion == {}:
            continue
        for word in token_frequency_present_2[emotion]:
            token_frequency_present[emotion][word] = token_frequency_present_2[emotion][word]
    return [token_frequency_present,token_frequency_absent_2]


#part 3 for checking the words in the synonyms:
def token_checker_3(token_frequency_present,token_frequency_absent_2,emotion_dict):    
    for word in token_frequency_absent_2:
        #create a list with all synonyms
        syns = wordnet.synsets(word)
        syns_words = []
        for n in range(len(syns)):
            syns_words.append(syns[n].lemmas()[0].name())
        #print(syns_words)
        # go in each word in syns_words to make comparison
        present = False        
        token_frequency_absent_3 = {}
        for word_s in syns_words:
            if present == True:
                break
                #check if it is token_frequency_present, 
                #if yes update the frequency and exit all the for loops except the first one
            if word_s is token_frequency_present:
                token_frequency_present[word_s] = token_frequency_present[word_s] + token_frequency_absent_2[word]
                present = True
                #print(present)
            else:
                #if is is absent in token_frequency_present, 
                #check emotion dictionary and if it is present exit all except first for
                for emotion in emotion_dict: # going into sad in the emotion dict
                    if word_s in emotion_dict[emotion]: # going into the list of words in sad words
                        token_frequency_present[emotion][word]= token_frequency_absent_2[word]
                        present = True
                        #print("present for", word_s)
        #if it is absent in emotion dictionary, add it to token_frequency_present_3
        if present == False:
            token_frequency_absent_3[word]= token_frequency_absent_2[word]
    return [token_frequency_present, token_frequency_absent_3]

def create_param(token_frequency_present,token_frequency):    
    parameter1 = {}
    for emotion in token_frequency_present:
        parameter1[emotion] = 0
    for emotion in token_frequency_present:
        parameter1[emotion] = sum(token_frequency_present[emotion].values())

    parameter2 = {}
    total_words = sum(token_frequency.values())
    for emotion in parameter1:
        parameter2[emotion] = parameter1[emotion]/total_words    
    return [parameter1, parameter2]
    
def token_checker(emotion_dict,token_frequency):
    [token_frequency_present,token_frequency_absent_1] = token_checker_1(emotion_dict,token_frequency)
    #print("part 1 token_frequency_present is", token_frequency_present, "\n")
    #print("part 1 token_frequency_absent is", token_frequency_absent_1,"\n")
    #print("part 1 params are", create_param(token_frequency_present,token_frequency))
    [token_frequency_present, token_frequency_absent_2] = token_checker_2(token_frequency_present,token_frequency_absent_1,emotion_dict)
    #print("part 2 token_frequency_present is", token_frequency_present,"\n")
    #print("part 2 token_frequency_absent is", token_frequency_absent_2, "\n")
    #print("part 2 params are", create_param(token_frequency_present,token_frequency))
    [token_frequency_present, token_frequency_absent_3] = token_checker_3(token_frequency_present,token_frequency_absent_2,emotion_dict)
    #print("part 3 token_frequency_present is", token_frequency_present, "\n")
    #print("part 3 token_frequency_absent is", token_frequency_absent_3, "\n")
    #print("part 3 params are", create_param(token_frequency_present,token_frequency))
    return create_param(token_frequency_present,token_frequency)
    #print(parameter1)
    #print(parameter2)

def article_parameters(articleName):
    token_frequency = clean_article(articleName)
    return token_checker(emotion_dict,token_frequency)

def predictEmotion(articlename):
    trust_testlist = []
    fear_testlist = []
    sadness_testlist = []
    anger_testlist = []
    surprise_testlist = []
    disgust_testlist = []
    joy_testlist = []
    anticipation_testlist = []
    
    trust_trainlist = []
    fear_trainlist = []
    sadness_trainlist = []
    anger_trainlist = []
    surprise_trainlist = []
    disgust_trainlist = []
    joy_trainlist = []
    anticipation_trainlist = []
    article_reader_list = []
    
    for i in range(25):
       article_reader_list.append("train_article" + str(i + 1) + ".txt")

    for article in article_reader_list:
        train_outputs = article_parameters(article)
        listofP1 = list(train_outputs[0].values())
        listofP2 = list(train_outputs[1].values())
        param_merge = [list(each_emot) for each_emot in zip(listofP1, listofP2)]

        trust_trainlist.append(param_merge[0])
        fear_trainlist.append(param_merge[1])
        sadness_trainlist.append(param_merge[2])
        anger_trainlist.append(param_merge[3])
        surprise_trainlist.append(param_merge[4])
        disgust_trainlist.append(param_merge[5])
        joy_trainlist.append(param_merge[6])
        anticipation_trainlist.append(param_merge[7])
    

    test_article = article_parameters(articlename)
    testlistofP1 = list(test_article[0].values())
    testlistofP2 = list(test_article[1].values())
    test_param_merge = [list(each_emot) for each_emot in zip(testlistofP1, testlistofP2)]

    trust_testlist.append(test_param_merge[0])
    fear_testlist.append(test_param_merge[1])
    sadness_testlist.append(test_param_merge[2])
    anger_testlist.append(test_param_merge[3])
    surprise_testlist.append(test_param_merge[4])
    disgust_testlist.append(test_param_merge[5])
    joy_testlist.append(test_param_merge[6])
    anticipation_testlist.append(test_param_merge[7])
    
    clf = SVR()
    sadnessfit = clf.fit(sadness_trainlist, train_sadness)
    trustfit = clf.fit(trust_trainlist, train_trust)
    fearfit = clf.fit(fear_trainlist, train_fear)
    angerfit = clf.fit(anger_trainlist, train_anger)
    surprisefit = clf.fit(surprise_trainlist, train_surprise)
    disgustfit = clf.fit(disgust_trainlist, train_disgust)
    joyfit = clf.fit(joy_trainlist, train_joy)
    anticipationfit = clf.fit(anticipation_trainlist, train_anticipation)
    
    sadval = sadnessfit.predict(sadness_testlist)
    trustval = trustfit.predict(trust_testlist)
    fearval = fearfit.predict(fear_testlist)
    angerval = angerfit.predict(anger_testlist)
    surpriseval = surprisefit.predict(surprise_testlist)
    disgustval = disgustfit.predict(disgust_testlist)
    joyval = joyfit.predict(joy_testlist)
    anticipationval = anticipationfit.predict(anticipation_testlist)
    
    print("Thank you for using EmotiFind by the Five Guys! For the article you have selected, the predicted emotions are:\n",
         "Sadness:", sadval[0], "\n", 
         "Anger:", angerval[0], "\n", 
          "Joy:", joyval[0], "\n",
          "Trust:", trustval[0], "\n",
         "Fear:", fearval[0], "\n",
         "Surprise:", surpriseval[0], "\n",
         "Disgust:", disgustval[0], "\n",
         "Anticipation:", anticipationval[0], "\n",
         "Thank you! Please try us again soon!")
    

In [18]:
test_reader_list = []

for i in range(25):
    test_reader_list.append("train_article" + str(i + 1) + ".txt")

for article in test_reader_list:
    predictEmotion(article)


Thank you for using EmotiFind by the Five Guys! For the article you have selected, the predicted emotions are:
 Sadness: -0.07433237432942524 
 Anger: 0.250053834601244 
 Joy: 0.09880697571931346 
 Trust: 0.23585118113484183 
 Fear: 0.09880697571931346 
 Surprise: 0.7489239132192381 
 Disgust: 0.09880697571931346 
 Anticipation: 0.10022281411770509 
 Thank you! Please try us again soon!
Thank you for using EmotiFind by the Five Guys! For the article you have selected, the predicted emotions are:
 Sadness: 0.39950707772081806 
 Anger: 0.7845356375399681 
 Joy: 0.2357051357532984 
 Trust: 0.24727227921377185 
 Fear: 0.0996744453537744 
 Surprise: 0.37323015883738736 
 Disgust: 0.24727173108234368 
 Anticipation: 0.10009121356669573 
 Thank you! Please try us again soon!
Thank you for using EmotiFind by the Five Guys! For the article you have selected, the predicted emotions are:
 Sadness: -0.07407170272543523 
 Anger: 0.10000166384595949 
 Joy: 0.24563659580119843 
 Trust: 0.400043541088

Thank you for using EmotiFind by the Five Guys! For the article you have selected, the predicted emotions are:
 Sadness: 0.2527388997575681 
 Anger: 0.09808323648552952 
 Joy: 0.27993928200168045 
 Trust: 0.2472722792137737 
 Fear: 0.2613723976957586 
 Surprise: 0.7841773606978507 
 Disgust: 0.1580055991050177 
 Anticipation: 0.09997703808157646 
 Thank you! Please try us again soon!
Thank you for using EmotiFind by the Five Guys! For the article you have selected, the predicted emotions are:
 Sadness: 0.22734140833195002 
 Anger: 0.24727173056261403 
 Joy: -0.07327859191698083 
 Trust: 0.373652478551318 
 Fear: 0.1579536219402039 
 Surprise: 0.1579863937507921 
 Disgust: 0.24727173056261403 
 Anticipation: 0.3998936717139161 
 Thank you! Please try us again soon!
Thank you for using EmotiFind by the Five Guys! For the article you have selected, the predicted emotions are:
 Sadness: 0.04689888434483591 
 Anger: 0.09797982225652957 
 Joy: 0.2356554646586633 
 Trust: 0.2472722792137737 
